# Multiwell Tile Display
### View multiple tile acquisition series in the tileviewer

In the following cell,

1. Edit the `output_directory` to be where we should build the 'ImagePyramid' multi-level image. Note that this directory is both a destination and a name (will be created if it doesn't exist) and *may become quite large* (larger than sum of ImagePyramids for individual tile series)

2. Edit the `series_glob_pattern` to find your series on the cluster. Alternatively, enter your series manually as a list in `tile_series`.

Run the cell, and if the expected number of output series is displayed, run the whole notebook.
Upon completion of the ImagePyramid a tileviewer will be opened in your default webbrowser.

### Notes
- we assume tile series are on a cluster, as '.pcs' files (spooled as pzf files)
- this computer must have PYME installed, and be connected to that cluster
- The concatenated datasource used in this will never be itself saved to disk, but the directories and files supporting the ImagePyramid we display in the tileviewer will be saved to `output_directory` and may be large (for example: using 96 series, each about 400 pixel diameter and spaced on a 9 mm grid [typical 96 well plate], the resulting directory takes up 15.5 GB)

In [ ]:
output_directory = '/media/smeagol/SSD/ultron_the_supersource'
series_glob_pattern = 'Bergamot/2020_11_3/EMC0406S1/try5/00*/*.pcs'

tile_series = ['pyme-cluster:///' + s for s in clusterIO.cglob(series_glob_pattern)]
print('%d tile series selected' % len(tile_series))

In [ ]:
from PYME.IO import clusterIO, MetaDataHandler
from PYME.Analysis import tile_pyramid
from PYME.IO.image import ImageStack
from PYME.IO.DataSources import ClusterPZFDataSource, BaseDataSource
import numpy as np
from PYME.Analysis.tile_pyramid import get_position_from_events
import subprocess
import sys
import time
import requests
import os

class ConcatenatedDataSource(BaseDataSource.BaseDataSource):
    moduleName = 'booyah'
    def __init__(self, datasource_list):
        self._datasources = datasource_list
        self._n_datasources = len(self._datasources)
        self._n_frames = np.zeros(self._n_datasources, dtype=int)
        
        for ind in range(self._n_datasources):
            # fixme - care about startat? all zero for my spooled series
            self._n_frames[ind] = self._datasources[ind].shape[2]
        
        self._n_frames_total = self._n_frames.sum()
        
        self._indices = np.zeros(self._n_frames_total, 
                                 dtype=[('series', int), ('frame', int)])
        self._positions = np.zeros(self._n_frames_total,
                                   dtype=[('x', float), ('y', float)])

        start = 0
        for ind in range(self._n_datasources):
            self._indices[start:start + self._n_frames[ind]]['series'] = ind 
            frames = np.arange(self._n_frames[ind], dtype=int)
            self._indices[start:start + self._n_frames[ind]]['frame'] = frames
            
            x_map, y_map = get_position_from_events(self._datasources[ind].getEvents(),
                                                    self._datasources[ind].mdh)
            self._positions['x'][start:start + self._n_frames[ind]] = x_map(frames)
            self._positions['y'][start:start + self._n_frames[ind]] = y_map(frames)
            start += self._n_frames[ind]
        
        self.mdh = MetaDataHandler.NestedClassMDHandler()
        self.mdh.update(self._datasources[0].mdh)
        
        self.fshape = None
    
    def getSlice(self, ind):
        ds_ind, frame_ind = self._indices[ind]['series'], self._indices[ind]['frame']
        return self._datasources[ds_ind].getSlice(frame_ind)

    def getSliceShape(self):
        if self.fshape is None:
            self.fshape = self.getSlice(0).shape
        return self.fshape
        
    def getNumSlices(self):
        return self._n_frames_total

    def getEvents(self):
        return []
        
    def getMetadata(self):
        return self.mdh

In [ ]:
# build up our hacked/mega datasource
datasources = []
for series in tile_series:
    datasources.append(ClusterPZFDataSource.DataSource(series))

ultron_the_supersource = ConcatenatedDataSource.DataSource(datasources)
im = ImageStack(ultron_the_supersource)

In [ ]:
tile_pyramid.tile_pyramid(output_directory, im.data, im.data._positions['x'], im.data._positions['y'], im.data.mdh, dark=im.data.mdh['Camera.ADOffset'])

In [ ]:

if not os.path.isabs(output_directory):
    # TODO - should we be doing the `.isabs()` check on the parent directory instead?
    from PYME.IO.FileUtils import nameUtils
    tiledir = nameUtils.getFullFilename(output_directory)

try:  # if we already have a tileviewer serving, change the directory
    requests.get('http://127.0.0.1:8979/set_tile_source?tile_dir=%s' % output_directory)
except requests.ConnectionError:  # start a new process
    try:
        pargs = {'creationflags': subprocess.CREATE_NEW_CONSOLE}
    except AttributeError:  # not on windows
        pargs = {'shell': True}
    
    self._gui_proc = subprocess.Popen('%s -m PYME.tileviewer.tileviewer %s' % (sys.executable, output_directory), **pargs)
    time.sleep(3)

# import webbrowser
# webbrowser.open('http://127.0.0.1:8979/')

In [ ]:
%%html
<iframe src="http://127.0.0.1:8979/" width="1200" height="1000"></iframe>